In [8]:
import tqdm as notebook_tqdm
from datasets import load_dataset

hf_dataset_identifier = "segments/sidewalk-semantic"

ds = load_dataset(hf_dataset_identifier)

ds = ds.shuffle(seed=42)
ds = ds["train"].train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]


In [9]:
import json
from huggingface_hub import hf_hub_download

repo_id = f"datasets/{hf_dataset_identifier}"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [10]:
from torchvision.transforms import ColorJitter
from transformers import SegformerFeatureExtractor

feature_extractor = SegformerFeatureExtractor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs

def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs

# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

/home/jgong/dev/conda/envs/ml-pytorch/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(


In [11]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.running_var', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.bias', 'decode_head.linear_fuse.weight', 'decode_head.classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments

epochs = 2 # 50 training for 2 epochs for the benchmarking
lr = 0.00006
batch_size = 2

hub_model_id = "hufanyoung/segformer-b0-finetuned-segments-sidewalk-2"

training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-sidewalk-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)


In [16]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    # currently using _compute instead of compute
    # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=feature_extractor.do_reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics


In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)


Cloning https://huggingface.co/hufanyoung/segformer-b0-finetuned-segments-sidewalk-2 into local empty directory.
Download file pytorch_model.bin:   0%|          | 6.25k/14.3M [00:00<?, ?B/s]












































Download file pytorch_model.bin: 100%|██████████| 14.3M/14.3M [00:01<00:00, 14.7MB/s]
Download file runs/Apr15_21-49-20_7ea360766af0/events.out.tfevents.1650059381.7ea360766af0.71.2: 100%|██████████| 9.82k/9.82k [00:01<?, ?B/s]


Download file runs/Apr15_21-49-20_7ea360766af0/1650059381.5713542/events.out.tfevents.1650059381.7ea360766af0.71.3: 100%|██████████| 4.86k/4.86k [00:01<?, ?B/s]




Clean file runs/Apr15_21-49-20_7ea360766af0/events.out.tfevents.1650059381.7ea360766af0.71.2: 100%|██████████| 9.82k/9.82k [00:01<00:00, 8.87kB/s]






Download file training_args.bin: 100%|██████████| 3.11k/3.11k [00:01<?, ?B/s]








Clean file runs/Apr15_21-49-20_7ea360766af0/1650059381.5713542/events.out.tfevents.1650059381.7ea360766af0.71.3: 100%|██████████| 4

In [18]:
trainer.train()


/home/jgong/dev/conda/envs/ml-pytorch/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/jgong/dev/conda/envs/ml-pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Unlabeled,Accuracy Flat-road,Accuracy Flat-sidewalk,Accuracy Flat-crosswalk,Accuracy Flat-cyclinglane,Accuracy Flat-parkingdriveway,Accuracy Flat-railtrack,Accuracy Flat-curb,Accuracy Human-person,Accuracy Human-rider,Accuracy Vehicle-car,Accuracy Vehicle-truck,Accuracy Vehicle-bus,Accuracy Vehicle-tramtrain,Accuracy Vehicle-motorcycle,Accuracy Vehicle-bicycle,Accuracy Vehicle-caravan,Accuracy Vehicle-cartrailer,Accuracy Construction-building,Accuracy Construction-door,Accuracy Construction-wall,Accuracy Construction-fenceguardrail,Accuracy Construction-bridge,Accuracy Construction-tunnel,Accuracy Construction-stairs,Accuracy Object-pole,Accuracy Object-trafficsign,Accuracy Object-trafficlight,Accuracy Nature-vegetation,Accuracy Nature-terrain,Accuracy Sky,Accuracy Void-ground,Accuracy Void-dynamic,Accuracy Void-static,Accuracy Void-unclear,Iou Unlabeled,Iou Flat-road,Iou Flat-sidewalk,Iou Flat-crosswalk,Iou Flat-cyclinglane,Iou Flat-parkingdriveway,Iou Flat-railtrack,Iou Flat-curb,Iou Human-person,Iou Human-rider,Iou Vehicle-car,Iou Vehicle-truck,Iou Vehicle-bus,Iou Vehicle-tramtrain,Iou Vehicle-motorcycle,Iou Vehicle-bicycle,Iou Vehicle-caravan,Iou Vehicle-cartrailer,Iou Construction-building,Iou Construction-door,Iou Construction-wall,Iou Construction-fenceguardrail,Iou Construction-bridge,Iou Construction-tunnel,Iou Construction-stairs,Iou Object-pole,Iou Object-trafficsign,Iou Object-trafficlight,Iou Nature-vegetation,Iou Nature-terrain,Iou Sky,Iou Void-ground,Iou Void-dynamic,Iou Void-static,Iou Void-unclear
20,2.768300,3.171063,0.085942,0.136347,0.594143,nan,0.425367,0.918091,0.001617,0.001761,0.003946,nan,0.000470,0.025641,0.000000,0.880821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000078,0.234857,0.000000,0.040310,0.002604,0.000000,nan,0.000000,0.083042,0.000000,0.000000,0.960732,0.084151,0.679110,0.000000,0.000000,0.020493,0.000000,0.000000,0.349773,0.637718,0.001589,0.001743,0.003870,nan,0.000464,0.017710,0.000000,0.395210,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000061,0.214766,0.000000,0.023958,0.002158,0.000000,0.000000,0.000000,0.029465,0.000000,0.000000,0.578248,0.072505,0.579119,0.000000,0.000000,0.013666,0.000000
40,2.559200,2.401905,0.091625,0.140696,0.629858,nan,0.569521,0.937952,0.000590,0.001260,0.008058,nan,0.000010,0.000000,0.000000,0.844238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.349027,0.000000,0.019593,0.000114,0.000000,nan,0.000000,0.016707,0.000000,0.000000,0.985233,0.042347,0.727295,0.000000,0.000000,0.000343,0.000000,0.000000,0.416701,0.666785,0.000586,0.001258,0.007925,nan,0.000009,0.000000,0.000000,0.493006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.305228,0.000000,0.018039,0.000102,0.000000,0.000000,0.000000,0.015439,0.000000,0.000000,0.522057,0.037089,0.630707,0.000000,0.000000,0.000333,0.000000
60,2.032400,1.987704,0.105863,0.158909,0.672657,nan,0.621367,0.942299,0.000445,0.014513,0.006183,nan,0.000000,0.000000,0.000000,0.886214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.661210,0.000000,0.003280,0.000020,0.000000,nan,0.000000,0.001091,0.000000,0.000000,0.960764,0.132950,0.854655,0.000000,0.000000,0.000091,0.000000,0.000000,0.442175,0.687083,0.000445,0.014491,0.006105,nan,0.000000,0.000000,0.000000,0.500478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.494995,0.000000,0.003209,0.000020,0.000000,0.000000,0.000000,0.001083,0.000000,0.000000,0.642263,0.112550,0.694363,0.000000,0.000000,0.000091,0.000000
80,1.881300,1.798102,0.121482,0.168806,0.689422,nan,0.704669,0.924858,0.000000,0.001656,0.004294,nan,0.000000,0.000000,0.000000,0.879187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.772755,0.000000,0.002765,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.939929,0.296693,0.874989,0.000000,0.000000,0.000000,0.000000,nan,0.451567,0.705295,0.000000,0.001655,0.004259,nan,0

/home/jgong/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/jgong/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/home/jgong/dev/conda/envs/ml-pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/jgong/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/

TrainOutput(global_step=400, training_loss=1.783346957564354, metrics={'train_runtime': 843.0153, 'train_samples_per_second': 1.898, 'train_steps_per_second': 0.474, 'total_flos': 2.81087582404608e+16, 'train_loss': 1.783346957564354, 'epoch': 2.0})